#### <strong>第三次作品：三個分類器於兩組影像的評比實驗</strong>
學號：711333104

姓名：林子齊
<hr>

 <font color=skyblue>作品目標</font>：
 <hr>

In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [9]:
data_file = 'Data/face_data.csv'
df = pd.read_csv(data_file)

# 顯示前幾筆資料
print(df.head())

# 顯示欄位名稱與資料型態
print(df.info())

# 若想看欄位名稱
print(df.columns)

# 顯示資料維度
print("資料維度：", df.shape)


          0         1         2         3         4         5         6  \
0  0.309917  0.367769  0.417355  0.442149  0.528926  0.607438  0.657025   
1  0.454545  0.471074  0.512397  0.557851  0.595041  0.640496  0.681818   
2  0.318182  0.400826  0.491736  0.528926  0.586777  0.657025  0.681818   
3  0.198347  0.194215  0.194215  0.194215  0.190083  0.190083  0.243802   
4  0.500000  0.545455  0.582645  0.623967  0.648760  0.690083  0.694215   

          7         8         9  ...      4087      4088      4089      4090  \
0  0.677686  0.690083  0.685950  ...  0.669422  0.652893  0.661157  0.475207   
1  0.702479  0.710744  0.702479  ...  0.157025  0.136364  0.148760  0.152893   
2  0.685950  0.702479  0.698347  ...  0.132231  0.181818  0.136364  0.128099   
3  0.404959  0.483471  0.516529  ...  0.636364  0.657025  0.685950  0.727273   
4  0.714876  0.723140  0.731405  ...  0.161157  0.177686  0.173554  0.177686   

       4091      4092      4093      4094      4095  target  
0  0.1

In [10]:
from HW3_common_utils import load_csv_data, split_and_scale

data_file = 'Data/face_data.csv'
X, y = load_csv_data(data_file)
X_train, X_test, y_train, y_test = split_and_scale(X, y, test_size=0.3)

print(f"Training samples: {X_train.shape[0]}")
print(f"Testing samples: {X_test.shape[0]}")
print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")
print("類別數量：", len(np.unique(y)))

Training samples: 280
Testing samples: 120
Training data shape: (280, 4096)
Testing data shape: (120, 4096)
類別數量： 40


In [16]:
data_file = 'Data/face_data.csv'
df = pd.read_csv(data_file)
X = np.array(df.iloc[:, :-1]) # 排除最後一欄標籤 N x p
y = np.array(df.iloc[:, -1])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, stratify=y)

scaler = StandardScaler()
X_train_ = scaler.fit_transform(X_train) # 標準化訓練資料
X_test_ = scaler.transform(X_test) # 標準化測試資料
# print the numbers of training and testing samples
print(f"Training samples: {X_train_.shape[0]}")
print(f"Testing samples: {X_test_.shape[0]}")
# print the shape of the data
print(f"Training data shape: {X_train_.shape}")
print(f"Testing data shape: {X_test_.shape}")
print("類別數量：", len( np.unique(y)))

Training samples: 280
Testing samples: 120
Training data shape: (280, 4096)
Testing data shape: (120, 4096)
類別數量： 40


In [ ]:
from HW3_common_utils import run_logistic_regression_cv

result = run_logistic_regression_cv(
    X_train, X_test, y_train, y_test,
    solver='lbfgs',
    Cs= np.logspace(-5, 5, 20),
    cv=5,
    tol=1e-6,
    max_iter=int(1e6),
    verbose=0,
    print_report=True
)

Logistic Regression with CV (solver = lbfgs)
Best C = [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1]
Training Accuracy: 100.00%
Testing Accuracy : 95.83%


In [22]:
from HW3_common_utils import apply_pca

X_train_pca, X_test_pca, pca = apply_pca(X_train, X_test, variance_ratio=0.8)

In [1]:
from importlib import reload
import HW3_common_utils
reload(HW3_common_utils)

<module 'HW3_common_utils' from 'c:\\Users\\Jimmy\\Desktop\\git\\ML711333104\\HW3_common_utils.py'>

In [23]:
from HW3_common_utils import run_logistic_regression_cv

result = run_logistic_regression_cv(
    X_train_pca, X_test_pca, y_train, y_test,
    solver='lbfgs',
    Cs=np.logspace(-5, 5, 20),
    cv=5,
    tol=1e-6,
    max_iter=int(1e6),
    verbose=0,
    print_report=True
)

Logistic Regression with CV (solver = lbfgs)
Best C = [1.83298071 1.83298071 1.83298071 1.83298071 1.83298071 1.83298071
 1.83298071 1.83298071 1.83298071 1.83298071 1.83298071 1.83298071
 1.83298071 1.83298071 1.83298071 1.83298071 1.83298071 1.83298071
 1.83298071 1.83298071 1.83298071 1.83298071 1.83298071 1.83298071
 1.83298071 1.83298071 1.83298071 1.83298071 1.83298071 1.83298071
 1.83298071 1.83298071 1.83298071 1.83298071 1.83298071 1.83298071
 1.83298071 1.83298071 1.83298071 1.83298071]
Training Accuracy: 100.00%
Testing Accuracy : 96.67%


In [11]:
from HW3_common_utils import run_logistic_gridcv

param_grid = {
    'solver': ['lbfgs', 'liblinear', 'saga'],
    'C': [0.1, 1, 10]
}

best_params, best_score, best_model = run_logistic_gridcv(
    X_train, y_train,
    param_grid=param_grid
)

{'C': 0.1, 'solver': 'lbfgs'}
0.9452380952380952
LogisticRegression(C=0.1, max_iter=1000000, tol=1e-06)
